# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [74]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path 

import holoviews as hv
import geoviews as gv
import hvplot.pandas  # Required for plotting with GeoViews
from geoviews import opts
# Import API key
from api_keys import geoapify_key

In [75]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../Output/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,montepuez,-13.1256,38.9997,27.11,46,0,3.33,MZ,1715328557
1,1,bilibino,68.0546,166.4372,0.11,97,100,1.07,RU,1715328559
2,2,isafjordur,66.0755,-23.1240,4.02,75,75,2.06,IS,1715328560
3,3,port-aux-francais,-49.3500,70.2167,7.68,88,100,19.89,TF,1715328560
4,4,tolanaro,-25.0319,46.9987,26.32,62,0,7.19,MG,1715328561


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [76]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,montepuez,-13.1256,38.9997,27.11,46,0,3.33,MZ,1715328557
1,1,bilibino,68.0546,166.4372,0.11,97,100,1.07,RU,1715328559
2,2,isafjordur,66.0755,-23.1240,4.02,75,75,2.06,IS,1715328560
3,3,port-aux-francais,-49.3500,70.2167,7.68,88,100,19.89,TF,1715328560
4,4,tolanaro,-25.0319,46.9987,26.32,62,0,7.19,MG,1715328561


In [86]:
humidity_data_df = pd.DataFrame({
    'city': city_data_df['City'],  
    'latitude': city_data_df['Lat'],  
    'longitude': city_data_df['Lng'],  
    'humidity': city_data_df['Humidity']  
})

# Configure the map plot
points = gv.Points(humidity_data_df, ['longitude', 'latitude'], ['city', 'humidity'])

map_overlay = gv.tile_sources.OSM * points.opts(
    opts.Points(size=10, cmap='Category20', color=hv.dim('city') ,tools=['hover'], legend_position='right', alpha=0.7)
)

# Display the map
map_overlay.opts(width=1200, height=600)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [78]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.head()
best_temp_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]
# Drop any rows with null values
best_temp_df.dropna()
# Display sample data
best_temp_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,tamanrasset,22.7850,5.5228,26.95,8,0,2.57,DZ,1715328620
91,91,pangai,-19.8000,-174.3500,25.58,57,0,4.43,TO,1715328669
119,119,sidi okba,34.7451,5.8983,22.17,38,0,3.39,DZ,1715328695
235,235,tossa de mar,41.7325,2.9338,21.59,33,0,2.60,ES,1715328808
241,241,ravar,31.2656,56.8055,22.75,25,0,3.57,IR,1715328813


### Step 3: Create a new DataFrame called `hotel_df`.

In [79]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_temp_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,tamanrasset,DZ,22.7850,5.5228,8,
91,pangai,TO,-19.8000,-174.3500,57,
119,sidi okba,DZ,34.7451,5.8983,38,
235,tossa de mar,ES,41.7325,2.9338,33,
241,ravar,IR,31.2656,56.8055,25,
310,balkanabat,TM,39.5108,54.3671,30,
327,san sebastian de la gomera,ES,28.0916,-17.1133,56,
442,lavra,PT,41.2594,-8.7185,67,
478,mahalapye,BW,-23.1041,26.8142,29,
518,belyy yar,RU,53.6039,91.3903,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [80]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter" : [],
    'categories' : 'accommodation.hotel',
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat'] 
    lon = row['Lng']   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address_rq = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_rq.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
tamanrasset - nearest hotel: فندق الأمان
pangai - nearest hotel: Billy's Place
sidi okba - nearest hotel: No hotel found
tossa de mar - nearest hotel: Aparthotel Tossa Park
ravar - nearest hotel: زائر سرا راور
balkanabat - nearest hotel: Nebitçi
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
lavra - nearest hotel: Singular
mahalapye - nearest hotel: Lme Lodge
belyy yar - nearest hotel: No hotel found
guanica - nearest hotel: Copamarina Beach Resort
<Response [200]>


,City,Country,Lat,Lng,Humidity,Hotel Name
55,tamanrasset,DZ,22.7850,5.5228,8,فندق الأمان
91,pangai,TO,-19.8000,-174.3500,57,Billy's Place
119,sidi okba,DZ,34.7451,5.8983,38,No hotel found
235,tossa de mar,ES,41.7325,2.9338,33,Aparthotel Tossa Park
241,ravar,IR,31.2656,56.8055,25,زائر سرا راور
310,balkanabat,TM,39.5108,54.3671,30,Nebitçi
327,san sebastian de la gomera,ES,28.0916,-17.1133,56,Hotel Villa Gomera
442,lavra,PT,41.2594,-8.7185,67,Singular
478,mahalapye,BW,-23.1041,26.8142,29,Lme Lodge
518,belyy yar,RU,53.6039,91.3903,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [87]:

hotel_data_df = pd.DataFrame({
    'city': hotel_df['City'],  
    'latitude': hotel_df['Lat'],  
    'longitude': hotel_df['Lng'],  
    'humidity': hotel_df['Humidity'],  
    'hotel name': hotel_df['Hotel Name'],
    'country' : hotel_df['Country']
})
# Configure the map plot
points = gv.Points(hotel_data_df, ['longitude', 'latitude'], ['city', 'humidity', 'hotel name', 'country'])

map_overlay = gv.tile_sources.OSM * points.opts(
    opts.Points(size=10, cmap='Category20', color=hv.dim('city') ,tools=['hover'], legend_position='right', alpha=0.7)
)

# Display the map
map_overlay.opts(width=1000, height=600)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,humidity,hotel name,country)